In [1]:
import pandas as pd

### 1. Import

In [2]:
df = pd.read_csv('data/cars_infos.csv')
df.columns

Index(['car_ID', 'symboling', 'CarName', 'fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg',
       'price'],
      dtype='object')

### 2. Traitement des colonnes

a. Indexion :<br>
- car_ID

In [3]:
# on supprimme la colonne et on garde l'index du dataframe
df = df.drop('car_ID', axis=1)

b. Séparation :  
- CarName

In [4]:
# on extrait le modèle inclut dans le CarName
df['Modèle'] = df.CarName.apply(lambda x : ' '.join(x.split(' ')[1:]).title())

# on ne garde que la marque dans le CarName
df.CarName = df.CarName.apply(lambda x : x.split(' ')[0].title())

# on renomme
df = df.rename(columns={'CarName':'Marque'})

df[['Marque','Modèle']].sample(5)

,Marque,Modèle
192,Volkswagen,Rabbit Custom
119,Plymouth,Cricket
142,Subaru,Brz
93,Nissan,Titan
175,Toyota,Corona


c. Traduction :  
- fueltype, carbody, drivewheel, enginelocation

In [5]:
# on définit les dictionnaires

fueltype_fr = {
    'gas' : 'Essence',
    'diesel' : 'Diesel'
}

carbody_fr = {
    'convertible':'Cabriolet',
    'hatchback':'Hayon',
    'sedan':'Berline',
    'wagon':'Break',
    'hardtop':'Coupé'
}

drivewheel_fr = {
    'rwd' : 'Propulsion arrière',
    'fwd' : 'Traction avant',
    '4wd' : 'Quatre roues motrices' 
}

enginelocation_fr = {
    'front':'Avant',
    'rear':'Arrière'
}

In [6]:
# On mappe les traductions grâce aux dictionnaires ci-dessus:
df.fueltype = df.fueltype.apply(lambda x : fueltype_fr[x])
df.carbody = df.carbody.apply(lambda x : carbody_fr[x])
df.drivewheel = df.drivewheel.apply(lambda x: drivewheel_fr[x])
df.enginelocation = df.enginelocation.apply(lambda x: enginelocation_fr[x])

# On traduit les colonnes
df = df.rename(columns={
    'fueltype':'Carburant',
    'carbody':'Classe',
    'drivewheel':'Transmission',
    'enginelocation':'PositionMoteur'
    })

df[['Carburant', 'Classe','Transmission','PositionMoteur']].sample(5)

,Carburant,Classe,Transmission,PositionMoteur
79,Essence,Hayon,Traction avant,Avant
96,Essence,Berline,Traction avant,Avant
85,Essence,Berline,Traction avant,Avant
7,Essence,Break,Traction avant,Avant
115,Essence,Berline,Propulsion arrière,Avant


d. Correspondance:  
- aspiration

In [7]:
#La colonne indique si la voiture possède un turbo
df.aspiration.unique()

array(['std', 'turbo'], dtype=object)

In [8]:
#On transforme la colonne en booléen Turbo = True/False
aspiration_fr = {
    'std' : False,
    'turbo' : True,
}
df.aspiration = df.aspiration.apply(lambda x : aspiration_fr[x])
df = df.rename(columns={'aspiration':'Turbo'})
df[['Turbo']].sample(5)

,Turbo
144,False
44,False
101,False
82,True
87,True


e. Numérisation:  
- doornumber, cylindernumber

In [9]:
# On définit le dictionnaire
number_fr = {
    'two' : 2,
    'three' : 3,
    'four' : 4,
    'five' : 5,
    'six' : 6,
    'eight' : 8,
    'twelve' : 12
}

In [10]:
#On transforme les valeurs textuelles en numérique grâce au dictionnaire:

df.doornumber = df.doornumber.apply(lambda x: number_fr[x]+1)
# /!\ On ajoutte un car en Français on compte le coffre comme porte

df.cylindernumber = df.cylindernumber.apply(lambda x: number_fr[x])

#on traduit les colonnes
df = df.rename(columns={
    'doornumber':'Portes',
    'cylindernumber':'Cylindres'})
df[['Portes','Cylindres']].sample(5)

,Portes,Cylindres
76,3,4
40,5,4
1,3,4
93,5,4
64,5,4


f. Conversion:  
- wheelbase, carlenght, carheight, curbweight, enginesize, boreratio, stroke, citympg, highwaympg

In [11]:
#on convertit de pouces(in) en centimètres(cm)
df.wheelbase = df.wheelbase.apply(lambda x: round(2.54*x))
df.carlength = df.carlength.apply(lambda x: round(2.54*x))
df.carwidth = df.carwidth.apply(lambda x: round(2.54*x))
df.carheight = df.carheight.apply(lambda x: round(2.54*x))
df.stroke = df.stroke.apply(lambda x: round(2.54*x))

#On convertit de livres(lbs) en tonnes(T)
df.curbweight = df.curbweight.apply(lambda x: round(0.45359*x)/1000)

#On convertit de pouces cubes(in³) en centimètres cubes(cm³)
df.enginesize = df.enginesize.apply(lambda x: round(16.3871*x))

#On convertit de pouces(in) en milimètres
df.boreratio = df.boreratio.apply(lambda x : round(25.4*x))

#On convertit de Miles per gallon(mpg) en litres au 100(L/100km)
df.citympg = df.citympg.apply(lambda x: round(x/235.214,3))
df.highwaympg = df.highwaympg.apply(lambda x: round(x/235.214,3))

#on traduit les colonnes en indiquant l'unité de mesure métrique
df = df.rename(columns={
    'wheelbase':'Empattement(cm)',
    'carlength':'Longueur(cm)',
    'carheight':'Hauteur(cm)',
    'carwidth' : 'Largeur(cm)',
    'curbweight' : 'Poids(t)',
    'enginesize': 'Moteur(cm³)',
    'boreratio' : 'Alésage(mm)',
    'stroke' : 'Piston(cm)',
    'citympg' : 'ConsommationVille(L/100km)',
    'highwaympg' : 'ConsommationAutoroute(L/100km)'
    })
df[['Empattement(cm)', 'Longueur(cm)', 'Hauteur(cm)', 'Poids(t)', 'Alésage(mm)', 'Piston(cm)', 'ConsommationVille(L/100km)', 'ConsommationAutoroute(L/100km)']].sample(5)

,Empattement(cm),Longueur(cm),Hauteur(cm),Poids(t),Alésage(mm),Piston(cm),ConsommationVille(L/100km),ConsommationAutoroute(L/100km)
33,238,381,134,0.880,74,9,0.128,0.145
141,247,437,133,0.973,92,7,0.136,0.157
144,246,437,138,1.082,92,7,0.102,0.106
28,262,443,152,1.150,85,9,0.102,0.128
66,266,444,138,1.225,87,9,0.132,0.166


g. Mise en forme:  
- enginetype, fuelsystem

In [12]:
# Le type de moteur et le système d'injection est representé par des codes suffisament éloquents
enginetype = df.enginetype.unique()
fuelsystem = df.fuelsystem.unique()
print(f'Engines types : {enginetype} \n Fuel systems : {fuelsystem}')

Engines types : ['dohc' 'ohcv' 'ohc' 'l' 'rotor' 'ohcf' 'dohcv'] 
 Fuel systems : ['mpfi' '2bbl' 'mfi' '1bbl' 'spfi' '4bbl' 'idi' 'spdi']


In [13]:
# On mets les codes en lettres capitales
df.enginetype = df.enginetype.apply(lambda x: x.upper())
df.fuelsystem = df.fuelsystem.apply(lambda x: x.upper())

# On renomme
df = df.rename(columns={
    'enginetype':'TypeMoteur',
    'fuelsystem':'Injecteur'
    })

df[['TypeMoteur','Injecteur']].sample(5)

,TypeMoteur,Injecteur
109,L,MPFI
162,OHC,2BBL
63,OHC,IDI
68,OHC,IDI
61,OHC,2BBL


h. Nom de colonnes:
- symboling, compressionratio, horsepower, peakrpm

In [14]:
# On traduit les colonnes restantes
df = df.rename(columns={
    'symboling' : 'CoteRisque',
    'compressionratio' : 'TauxCompression',
    'horsepower' : 'Chevaux',
    'peakrpm' : 'Régime(tr/min)',
    'price' : 'Prix($)',
})

i. Repositionnement des colonnes

In [15]:
# On reposition les colonnes en les regroupant par catégories
df = df[[
    'Prix($)', #Notre target
    'Marque', 'Modèle', 'Classe', 'Portes', #Caracteristiques physiques
    'Longueur(cm)', 'Hauteur(cm)', 'Largeur(cm)','Empattement(cm)', 'Moteur(cm³)','Poids(t)', #Dimensions
    'Carburant', 'ConsommationVille(L/100km)', 'ConsommationAutoroute(L/100km)', #Consommation
    'PositionMoteur','Transmission', #Mécanique
    'Chevaux', 'Régime(tr/min)', 'Turbo', 'Cylindres', #Puissance
    'Injecteur','TypeMoteur', #Codes
    'Alésage(mm)','Piston(cm)', #Pièces
    'TauxCompression', #Ratio
    'CoteRisque' #Assurance
    ]]

### 3. Traitement des lignes

a. Marque

In [16]:
# Certaines marques ont été mal orthographiées
df.Marque.unique()

array(['Alfa-Romero', 'Audi', 'Bmw', 'Chevrolet', 'Dodge', 'Honda',
       'Isuzu', 'Jaguar', 'Maxda', 'Mazda', 'Buick', 'Mercury',
       'Mitsubishi', 'Nissan', 'Peugeot', 'Plymouth', 'Porsche',
       'Porcshce', 'Renault', 'Saab', 'Subaru', 'Toyota', 'Toyouta',
       'Vokswagen', 'Volkswagen', 'Vw', 'Volvo'], dtype=object)

In [17]:
# On corrige en appliquant le bon orthographe sur des masques
df.loc[df.Marque == 'Alfa-Romero', 'Marque'] = 'Alfa-Romeo'
df.loc[df.Marque == 'Bmw', 'Marque'] = 'BMW'
df.loc[df.Marque == 'Toyouta', 'Marque'] = 'Toyota'
df.loc[df.Marque == 'Maxda', 'Marque'] = 'Mazda'
df.loc[df.Marque == 'Porcshce', 'Marque'] = 'Porsche'
df.loc[(df.Marque == 'Vokswagen') | (df.Marque == 'Vw'), 'Marque'] = 'Volkswagen'

In [18]:
# Toutes les marques sont orthographiées pareil
df.Marque.unique()

array(['Alfa-Romeo', 'Audi', 'BMW', 'Chevrolet', 'Dodge', 'Honda',
       'Isuzu', 'Jaguar', 'Mazda', 'Buick', 'Mercury', 'Mitsubishi',
       'Nissan', 'Peugeot', 'Plymouth', 'Porsche', 'Renault', 'Saab',
       'Subaru', 'Toyota', 'Volkswagen', 'Volvo'], dtype=object)

b. Modèle

In [19]:
# On mets le même orthographe au modèle orthographié "100Ls" et "100 Ls"
df.loc[df.Modèle == '100 Ls', 'Modèle'] = '100Ls'
df.Modèle.unique()

array(['Giulia', 'Stelvio', 'Quadrifoglio', '100Ls', 'Fox', '5000',
       '4000', '5000S (Diesel)', '320I', 'X1', 'X3', 'Z4', 'X4', 'X5',
       'Impala', 'Monte Carlo', 'Vega 2300', 'Rampage', 'Challenger Se',
       'D200', 'Monaco (Sw)', 'Colt Hardtop', 'Colt (Sw)',
       'Coronet Custom', 'Dart Custom', 'Coronet Custom (Sw)', 'Civic',
       'Civic Cvcc', 'Accord Cvcc', 'Accord Lx', 'Civic 1500 Gl',
       'Accord', 'Civic 1300', 'Prelude', 'Civic (Auto)', 'Mu-X',
       'D-Max ', 'D-Max V-Cross', 'Xj', 'Xf', 'Xk', 'Rx3', 'Glc Deluxe',
       'Rx2 Coupe', 'Rx-4', '626', 'Glc', 'Rx-7 Gs', 'Glc 4',
       'Glc Custom L', 'Glc Custom', 'Electra 225 Custom',
       'Century Luxus (Sw)', 'Century', 'Skyhawk', 'Opel Isuzu Deluxe',
       'Skylark', 'Century Special', 'Regal Sport Coupe (Turbo)',
       'Cougar', 'Mirage', 'Lancer', 'Outlander', 'G4', 'Mirage G4',
       'Montero', 'Pajero', 'Versa', 'Gt-R', 'Rogue', 'Latio', 'Titan',
       'Leaf', 'Juke', 'Note', 'Clipper', 'Nv200', '

### 4. Export


In [20]:
df.to_csv('data/cars_infos_cleaned.csv', index=False)